In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#import useful libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from random import randint

In [4]:
data = pd.read_csv('bank_transactions.csv', delimiter=',')

In [5]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Finding Similar Customers

## 1.1 Set up the date

clean the dataset

In [6]:
#check for any NA values
any(data.isna())

True

In [7]:
data.dropna(inplace=True)

convert string to datetime

In [8]:
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

data.drop(data[data.CustomerDOB.dt.year > 1998].index, axis = 0, inplace = True)

data.drop(data[data.CustomerDOB == 1800].index, axis = 0, inplace=True)

data = data.reset_index()

In [9]:
data

,index,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0
1,2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0
2,3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0
3,4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5
4,5,T6,C1536588,1972-08-10,F,ITANAGAR,53609.20,2016-02-08,173940,676.0
...,...,...,...,...,...,...,...,...,...,...
965271,1048562,T1048563,C8020229,1990-08-04,M,NEW DELHI,7635.19,2016-09-18,184824,799.0
965272,1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,183734,460.0
965273,1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0
965274,1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0


We are going to keep only the rows We are going to use later in the query. Thus, we are going to get rid of *TransactionID* and *CustomerID*

In [10]:
data = data[['CustomerDOB','CustGender','CustLocation','CustAccountBalance','TransactionDate','TransactionTime','TransactionAmount (INR)']]

First thing to do is convert our values in int32 dtype in order to apply the hash function

In [11]:
#let's define some functions
def float_to_int(value):
    integer,decimal  = str(value).split('.')
    return int(integer+decimal)

def date_to_int(value):
    return int(pd.Timestamp(value).timestamp())

def string_to_int(value):
    return sum(ord(x) for x in value)

In [12]:
data.CustomerDOB = data['CustomerDOB'].apply(date_to_int)
data.CustGender = data['CustGender'].apply(string_to_int)
data.CustLocation = data['CustLocation'].apply(string_to_int)
data.CustAccountBalance = data['CustAccountBalance'].apply(float_to_int)
data.TransactionDate = data['TransactionDate'].apply(date_to_int)
data['TransactionAmount (INR)'] = data['TransactionAmount (INR)'].apply(float_to_int)


In [13]:
data

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,780969600,70,755,1781905,1454889600,143207,250
1,848966400,70,443,1787444,1454889600,142712,4590
2,116812800,70,443,86650321,1454889600,142714,20600
3,575164800,70,777,671443,1454889600,181156,17625
4,82252800,70,583,536092,1454889600,173940,6760
...,...,...,...,...,...,...,...
965271,649728000,77,624,763519,1474156800,184824,7990
965272,698544000,77,446,2731142,1474156800,183734,4600
965273,611452800,77,644,22175706,1474156800,183313,7700
965274,273283200,77,968,1011787,1474156800,184706,10000


In [14]:
#obtain max value for x
data.max()

CustomerDOB                 913507200
CustGender                         84
CustLocation                     2609
CustAccountBalance         5536968816
TransactionDate            1481241600
TransactionTime                235959
TransactionAmount (INR)     156003499
dtype: int64

In [15]:
#let's get the next prime number bigger than x

from sympy import nextprime

max_value_of_x = nextprime(5536968816)

## 1.2 Fingerprint Hashing

The hash function we want to implement is in the following form: <br>
$\begin{equation} h(x)=(ax+b)\mod{5}\end{equation}$ <br>
where: <br>
- *x*: input value
- *a*, *b*: randomly choosen integers less than the maximum value of *x*
- *c*: the next prime number bigger than the maximum value of *x*

In [16]:
def hash(val, a, b, c):
    return (a*val+b)%c

Now it's time to implement the MinHash

In [17]:
def min_hash(info,c ,list_of_tuples):

    signatures = []

    for idx, x in enumerate(info):
        hash_value = []
        for values in list_of_tuples:
            a,b = values
            ans = hash(x,a,b,c,)
            hash_value.append(ans)
        signatures.append(int(min(hash_value)))
        
    return signatures

In [18]:
def random_coef(n_hash, M):

    a = []
    b = []

    for _ in range(n_hash):
        a_i = randint(0,M)
        a.append(a_i)
        b_i = randint(0,M)
        b.append(b_i)
    return a,b

In [19]:
c = max_value_of_x
M = 2**32 -1
n_hash = 10

a,b = random_coef(n_hash, M)

list_of_tuples = list(
    map(
        lambda x, y: (x, y),
        a,
        b
    )
)

data['min-hash']  = [min_hash([*x[1]],c,list_of_tuples) for x in (data.iterrows())]

In [20]:
data

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),min-hash
0,780969600,70,755,1781905,1454889600,143207,250,"[138285256, 1000127052, 357407960, 532723775, ..."
1,848966400,70,443,1787444,1454889600,142712,4590,"[698514112, 1000127052, 234306016, 1117416769,..."
2,116812800,70,443,86650321,1454889600,142714,20600,"[467976898, 1000127052, 234306016, 619728371, ..."
3,575164800,70,777,671443,1454889600,181156,17625,"[36065862, 1000127052, 4569222, 30085447, 1198..."
4,82252800,70,583,536092,1454889600,173940,6760,"[84041165, 1000127052, 709647149, 114984271, 1..."
...,...,...,...,...,...,...,...,...
965271,649728000,77,624,763519,1474156800,184824,7990,"[746569109, 18698871, 122167375, 83289677, 709..."
965272,698544000,77,446,2731142,1474156800,183734,4600,"[740213705, 18698871, 339521236, 5730347, 7099..."
965273,611452800,77,644,22175706,1474156800,183313,7700,"[299507120, 18698871, 115828072, 28209681, 709..."
965274,273283200,77,968,1011787,1474156800,184706,10000,"[1031505968, 18698871, 371570343, 927070640, 7..."


## 1.3 Locality Sensitive Hashing (LSH)

Now let's apply the same idea on the query, and then apply the Jaccard similarity since MinHash is stricly related to it

In [35]:
query = pd.read_csv('query_users.csv')

In [37]:
query

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,27/7/78,M,DELHI,94695.61,2/9/16,140310,65.00
1,6/11/92,M,PANCHKULA,7584.09,2/9/16,120214,6025.00
2,14/8/91,M,PATNA,7180.60,10/8/16,221732,541.50
3,3/1/87,M,CHENNAI,56847.75,29/8/16,144138,1000.00
4,4/1/95,M,GURGAON,84950.13,25/9/16,233309,80.00
5,10/1/81,M,WORLD TRADE CENTRE BANGALORE,23143.95,11/9/16,192906,303.00
6,20/9/76,F,CHITTOOR,15397.80,28/8/16,92633,20.00
7,10/4/91,M,MOHALI,426.30,2/8/16,203754,50.00
8,19/3/90,M,MOHALI,4609.34,26/8/16,184015,300.00
9,19/12/70,M,SERAMPORE,6695988.46,27/8/16,144030,299.00
